In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score, f1_score, confusion_matrix, precision_score, recall_score, precision_recall_curve, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight

y = screentime_data['Mental Health Treatment']
X = screentime_data[['screen_time','age']]

rnd = 10

train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = rnd, stratify=y)

class_weights = {'Does Not Need Treatment': 1, 'Needs Treatment': 5} 
sample_weights = compute_sample_weight(class_weights, train_y)
# sample_weights = compute_sample_weight('balanced', train_y)


forest_model = RandomForestClassifier(random_state=rnd, n_estimators=200)
# forest_model = DecisionTreeClassifier()
forest_model.fit(train_X, train_y, sample_weight=sample_weights)
mht_preds = forest_model.predict(val_X)
mht_probs = forest_model.predict_proba(val_X)[:, 1]  # Probabilities for class 1

print("ROC AUC Score:", roc_auc_score(val_y, mht_probs))
print("F1 Score:", f1_score(val_y, mht_preds, pos_label='Needs Treatment'))
print("Confusion Matrix:\n", confusion_matrix(val_y, mht_preds))
print("Precision:", precision_score(val_y, mht_preds, pos_label='Needs Treatment'))
print("Recall:", recall_score(val_y, mht_preds, pos_label='Needs Treatment'))

probs = forest_model.predict_proba(val_X)[:, 1]
precision, recall, thresholds = precision_recall_curve(val_y, probs, pos_label='Needs Treatment')

plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel('Threshold')
plt.legend()
plt.title('Precision-Recall Tradeoff')
plt.show()

plt.figure()

ConfusionMatrixDisplay.from_estimator(forest_model, val_X, val_y, normalize='true')

plt.show()

NameError: name 'screentime_data' is not defined